In [ ]:
!pip install trl --upgrade
%pip install datasets==2.16.0

In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from trl import RewardTrainer, RewardConfig
from datasets import Dataset

In [2]:
ds = load_dataset("nvidia/HelpSteer2", data_dir="preference")['train'] # despite the name, this contains both train and val, which you can use split to distinguish

In [3]:
ds = ds.filter(lambda x: x["split"] == "train")

In [4]:
train = {'chosen': [], 'rejected': []}
for x in ds:
    if x['preference_strength'] <= 0:
        train['chosen'].append(x['response_1'])
        train['rejected'].append(x['response_2'])
    else:
        train['rejected'].append(x['response_1'])
        train['chosen'].append(x['response_2'])

In [5]:
PATH = "/kaggle/input/sft-opt/sft_opt/"

tokenizer = AutoTokenizer.from_pretrained(PATH, local_files_only=True)
model = AutoModelForSequenceClassification.from_pretrained(PATH, local_files_only=True)
model.to('cuda')

Some weights of OPTForSequenceClassification were not initialized from the model checkpoint at /kaggle/input/sft-opt/sft_opt/ and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


OPTForSequenceClassification(
  (model): OPTModel(
    (decoder): OPTDecoder(
      (embed_tokens): Embedding(50272, 512, padding_idx=1)
      (embed_positions): OPTLearnedPositionalEmbedding(2050, 1024)
      (project_out): Linear(in_features=1024, out_features=512, bias=False)
      (project_in): Linear(in_features=512, out_features=1024, bias=False)
      (layers): ModuleList(
        (0-23): 24 x OPTDecoderLayer(
          (self_attn): OPTSdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (activation_fn): ReLU()
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Line

In [6]:
train_dataset = Dataset.from_dict(train)

In [7]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [8]:
reward_config = RewardConfig(
    learning_rate=1e-5, fp16=True, num_train_epochs=1, save_strategy="no", # it did not save
    save_total_limit=1, logging_steps=50, output_dir="/kaggle/working/", per_device_train_batch_size = 8)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [9]:
trainer = RewardTrainer(
    model=model,
    train_dataset=train_dataset,
    args=reward_config, 
    processing_class=tokenizer,
    max_length=512,
    )

trainer.train()

/opt/conda/lib/python3.10/site-packages/trl/trainer/reward_trainer.py:154: FutureWarning: The `max_length` argument is deprecated and will be removed in a future version. Please use the `RewardConfig` to set `max_length` instead.
  warnings.warn(


Map:   0%|          | 0/8677 [00:00<?, ? examples/s]

Map:   0%|          | 0/8677 [00:00<?, ? examples/s]

Filter:   0%|          | 0/8677 [00:00<?, ? examples/s]

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss
50,0.791000
100,0.731800
150,0.712800
200,0.681900
250,0.659500
300,0.677000
350,0.649100
400,0.699700
450,0.665600
500,0.630200


TrainOutput(global_step=815, training_loss=0.6751884554061421, metrics={'train_runtime': 1839.5088, 'train_samples_per_second': 3.543, 'train_steps_per_second': 0.443, 'total_flos': 0.0, 'train_loss': 0.6751884554061421, 'epoch': 1.0})

In [10]:
trainer.save_model('RM_opt')